Volvemos a cargar el dataset y una serie de otras cosas. 

Basado en este excelente [tutorial](https://keras.io/examples/graph/gnn_citations/).

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

citations = pd.read_csv("cora/cora.cites",
    sep="\t",
    header=None,
    names=["target", "source"],
)

column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv("cora/cora.content", sep="\t", header=None, names=column_names,
)


In [2]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

train_data, test_data = [], []

for _, group_data in papers.groupby("subject"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.5
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (1367, 1435)
Test data shape: (1341, 1435)


In [5]:
def create_MLP(hidden_layers, dropout_rate, name=None):
    model = nn.Sequential()
    num_layer = 0

    for layer in hidden_layers:
        num_layer +=1
        neurons_in = layer[0]
        neurons_out = layer[1]
        
        model.add_module("batchnorm"+str(num_layer), nn.BatchNorm1d(neurons_in))
        model.add_module("dropout"+str(num_layer), nn.Dropout(dropout_rate))
        model.add_module("dense"+str(num_layer), nn.Linear(neurons_in, neurons_out))
        model.add_module("activacion"+str(num_layer), nn.GELU())
    
    return(model)
        


## Manejo de datos específico para nuestras GNNs.

Lo primero es que ahora las GNNs van a funcionar en base a las conexiones entro los papers (además de los features obviamente). La GNN se compila con la info del grado, por lo que el x_train y x_test solo deben tener los id de los nodos relevantes. 

In [7]:
feature_names = list(set(papers.columns) - {"paper_id", "subject"})
num_features = len(feature_names)
num_classes = len(class_idx)

# Create train and test features as a numpy array.
x_train = train_data["paper_id"].to_numpy()
x_test = test_data["paper_id"].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["subject"]
y_test = test_data["subject"]

El segundo paso es crear una matriz de adyacencia en formato numpy, que es lo que vamos a necesitar para pasarselo a torch. Por razones de formato, es mejor usar una representación esparsa, en forma de lista de pares. 

In [10]:
#Matriz en forma de lista de pares
edges = citations[["source", "target"]].to_numpy().T

#Codigo para agregar peso a cada arista, por ahora son puros 1s, todas valen lo mismo. 
edge_weights = torch.ones(edges.shape[1])

# Crear (en formato torch) los features para cada nodo.
node_features = papers.sort_values("paper_id")[feature_names].to_numpy()
node_features = torch.tensor(node_features, dtype=torch.float32)

# el grafo es la union de estas tres cosas
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)


### Esto es muy importante. 
### El primer vector es la lista de los indices de los nodos source de edges, 
### El segundo vector es la lista de los indices de los nodos target

node_indices, neighbour_indices = edges[0], edges[1]

Edges shape: (2, 5429)
Nodes shape: torch.Size([2708, 1433])


### Un modelo para una capa de la GNN

Esta es la capa que va a hacer los pasos de agregación y update.  

In [ ]:
class GNNLayer(nn.Module):
    def __init__(self, input_features, capas_internas=[32, 32], dropout_rate=0.2, normalize=False):
        super(GNNLayer, self).__init__()

        #Hay dos redes neuronales involucradas en una capa de GNN: la primera activación de los mensajes, 
        #    y el manejo del update. 
        
        self.preprocessor = create_MLP([[input_features,hidden_layer_neurons],[hidden_layer_neurons,hidden_layer_neurons]], dropout_rate)
        self.updater = create_MLP([[hidden_layer_neurons,hidden_layer_neurons],[hidden_layer_neurons,hidden_layer_neurons]], dropout_rate)

    def prepare(self, node_representations, weights=None):
        
        #Esta funcion pasa los mensajes por una red neuronal simple, y aplica los pesos (si hay)
        
        messages = self.preprocessor(node_representations)
        if weights is not None:
            messages = messages * weights.unsqueeze(-1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_representations):
        # Esta funcion agrega los mensajes de cada nodo, en forma de suma. 
        # recibo un vector node_indices, que es de largo [num_edges] y me dice los nodos origen de cada arista
        # matriz neighbour_messages es de forma [num_edges, (neuronas_internas)], osea [num_edges, 32] en este codigo
        # esta matriz tiene el mensaje de cada nodo que participa en la arista como nodo destino
        # la matriz node_repesentations es de la forma [num_nodes, representation_dim], contiene información de 
        # los nodos del grafo. 

        num_nodes = node_repesentations.shape[0]

        #### JUAN: POR PONER
        
        aggregated_message = torch.zeros((num_nodes, neighbour_messages.shape[1])).to(neighbour_messages.device)
        aggregated_message = aggregated_message.scatter_add_(0, node_indices.unsqueeze(-1).expand_as(neighbour_messages), neighbour_messages)

    def update(self, node_representations, aggregated_messages):
        # Para combinar los mensajes con los features de cada nodo, concatenamos. 
        # Notar que a este punto tanto node_repesentations como aggregated_messages tienen forma 
        # [num_nodes, representation_dim]. Cat los concatena. 

        h = torch.cat([node_representations, aggregated_messages], dim=1)

        # Y aplicamos unas capas no-lineales
        
        node_embeddings = self.updater(h)

        return node_embeddings

    def forward(self, inputs):
        ## Procesa los inputs para crear los embeddings. Siempre tenemos información de todo el grafo, 
        ## y operamos sobre todos los nodos en node_representations
        
        node_representations, edges, edge_weights = inputs
        node_indices, neighbour_indices = edges

        # Lo primero es una lista de vectores en donde tomo cada id en neighbour_indices 
        # y lo reemplazo por la representación de ese id. 
        # El resultado es una lista que contiene, para cada arista, la representación del target de esa arista
        neighbour_representations = node_representations[neighbour_indices]

        # Procesamos estos mensajes (posiblemente incluyendo pesos en aristas)
        neighbour_messages = self.prepare(neighbour_representations, edge_weights)
        
        # Los agregamos
        aggregated_messages = self.aggregate(node_indices, neighbour_messages, node_representations)
        
        # Y finalmente el update
        return self.update(node_representations, aggregated_messages)



In [ ]:
Ahora juntamos varias capas! 

In [ ]:
class GNNClassifier(nn.Module):
    def __init__(self, graph_info, num_classes, input_features, hidden_layer_neurons=32, dropout_rate=0.2, normalize=True):
        super(GNNClassifier, self).__init__()

        #LA GNN maneja información de todo el grafo, independiente del batch que procese. 

        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        
        #normalizar
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        #Las layers básicas: una capa para preprocesar todo 
        self.preprocessor = create_MLP([[self.node_features,hidden_layer_neurons],[hidden_layer_neurons,hidden_layer_neurons]], dropout_rate)

        # dos capas de paso de mensjaes 
        
        self.layer1 = GNNLayer(hidden_layer_neurons, hidden_layer_neurons, dropout_rate=0.2, normalize=False)
        self.layer2 = GNNLayer(hidden_layer_neurons, hidden_layer_neurons, dropout_rate=0.2, normalize=False)

        # Un capa final.
        
        self.postprocesado = create_MLP([[hidden_layer_neurons,hidden_layer_neurons],[hidden_layer_neurons,hidden_layer_neurons]], dropout_rate)
        
        self.classifier = nn.Linear(hidden_layer_neurons, num_classes)

    def forward(self, batch_indices):
        
        #### Feature preprocessing layer to reduce dimensionality
        nodos_preprocesados = self.preprocessor(self.node_features)

        #### These preprocessed nodes go through capa1, which aggregates messages from their neighbors.
        paso_mens1 = self.capa1((nodos_preprocesados, self.edges, self.edge_weights))

        skip1 = nodos_preprocesados + paso_mens1

        paso_mens2 = self.capa2((skip1, self.edges, self.edge_weights))

        skip2 = paso_mens2 + skip1

        ##### Postprocessing to get to the node category
        postprocesado = self.postprocess(skip2)

        ##### Put embeddings back in the order required by the batch
        node_embeddings = postprocesado[batch_indices]

        # Readout to get to the categories
        return self.clas(node_embeddings)